# STEP 1: Loading Data

In [1]:
# Import dependencies
import pandas as pd
import os
import numpy as np

In [2]:
# Read the data into a Pandas DataFrame
crime_incident_df = pd.read_csv('Resources/CMPD_Incidents.csv')
crime_incident_df.head(10)

/var/folders/z3/mxhxwwgj7bb1sd23lpfhh6h80000gn/T/ipykernel_20039/1522160804.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_incident_df = pd.read_csv('Resources/CMPD_Incidents.csv')


,X,Y,YEAR,INCIDENT_REPORT_ID,LOCATION,CITY,STATE,ZIP,X_COORD_PUBLIC,Y_COORD_PUBLIC,...,LOCATION_TYPE_DESCRIPTION,PLACE_TYPE_DESCRIPTION,PLACE_DETAIL_DESCRIPTION,CLEARANCE_STATUS,CLEARANCE_DETAIL_STATUS,CLEARANCE_DATE,HIGHEST_NIBRS_CODE,HIGHEST_NIBRS_DESCRIPTION,OBJECTID,GlobalID
0,1.429778e+06,550302.999980,2023,20230817-1203-00,3100 TIMBERBROOK DR,CHARLOTTE,NC,28208.0,1429778,550303,...,Indoors,Residential,Apartment/Duplex Private Res,Open,Open,NaN,801,Suicide,1,{E6F8E379-522C-49CC-9093-3A1D2BA3023B}
1,1.450599e+06,467980.999927,2022,20220206-1938-01,10900 SEDGEMOOR LN,CHARLOTTE,NC,NaN,1450599,467981,...,Outdoors,Residential,Private Residence,Exceptionally Cleared,Located (Missing Persons and Runaways only),2022/02/08 00:00:00+00,800,Missing Person,2,{840550E3-41A4-4ECA-88A0-DBB2461FB9B6}
2,1.467768e+06,525875.999950,2023,20231218-1423-00,4300 CASTLETON RD,CHARLOTTE,NC,28211.0,1467768,525876,...,Indoors,Residential,Private Residence,Exceptionally Cleared,Cleared by Other Means,2024/02/08 00:00:00+00,899,Other Unlisted Non-Criminal,3,{9366A473-3328-40E6-A6DE-0F48476D3671}
3,1.403955e+06,498535.999840,2020,20201011-1424-00,10000 CLOISTERS CLUB LN,CHARLOTTE,NC,28278.0,1403955,498536,...,Parking Lot,Residential,Apartment/Duplex Private Res,Open,Open,NaN,23G,Theft of Motor Vehicle Parts from Vehicle,4,{4729C8A8-A93A-4E2C-8844-CB63E120681A}
4,1.430743e+06,518463.000012,2017,20170401-1213-00,700 DUNBROOK LN,CHARLOTTE,NC,28217.0,1430743,518463,...,Indoors,Residential,Private Residence,Exceptionally Cleared,Victim Chose not to Prosecute,2019/07/30 00:00:00+00,13C,Intimidation,5,{08DBBF46-7606-4A7D-A584-AA4091CD08A4}
5,1.450988e+06,586888.999971,2024,20240113-0129-04,9400 STATESVILLE RD,CHARLOTTE,NC,NaN,1450988,586889,...,Indoors,Commercial Place,Hotel/Motel,Cleared by Arrest by Another Agency,Cleared by Arrest by Another Agency,2024/11/13 00:00:00+00,13A,Aggravated Assault,6,{1D3A75BE-FCBC-4B7F-887E-231174BC6928}
6,1.475987e+06,528626.000027,2017,20170802-1513-00,5500 AMITY PL,CHARLOTTE,NC,28212.0,1475987,528626,...,Outdoors,Residential,Public Housing Private,Unfounded,Unfounded,2017/08/04 00:00:00+00,23F,Theft From Motor Vehicle,7,{CDDB3F02-CDCC-44AF-8D41-A488771365BD}
7,1.432367e+06,543043.000065,2023,20230114-2030-03,2500 CRISPIN AV,CHARLOTTE,NC,NaN,1432367,543043,...,Indoors,Residential,Private Residence,Cleared by Arrest,Cleared by Arrest,2023/01/20 00:00:00+00,13A,Aggravated Assault,8,{11966F88-8EBD-416C-8E5F-7C6563043132}
8,1.446605e+06,472993.000073,2017,20170829-1949-03,15800 MARVIN RD,CHARLOTTE,NC,28270.0,1446605,472993,...,Outdoors,Public/Non-Residential,Church/Synagogue/Temple,Cleared by Arrest,Cleared by Arrest,2017/09/02 00:00:00+00,90J,Trespass Of Real Property,9,{D88BFF74-FDE6-4EFE-96A1-AE7A2FAE996F}
9,1.426206e+06,514456.000152,2022,20221012-1737-01,2000 LONGDALE DR,CHARLOTTE,NC,NaN,1426206,514456,...,Outdoors,Residential,Private Residence,Exceptionally Cleared,Cleared by Other Means,2022/10/28 00:00:00+00,803,Overdose,10,{121D63E7-20CD-454E-8210-E024B45835BF}


# Converting to datime format

In [16]:
# Convert 'DATE_REPORTED' to datetime format
crime_incident_df['DATE_REPORTED'] = pd.to_datetime(crime_incident_df['DATE_REPORTED'], errors='coerce')


In [17]:
# Check for missing or invalid dates
missing_dates = crime_incident_df['DATE_REPORTED'].isna().sum()
print(f"Number of invalid/missing dates: {missing_dates}")

# Drop rows with invalid dates
crime_incident_df = crime_incident_df.dropna(subset=['DATE_REPORTED'])


Number of invalid/missing dates: 0


In [18]:
# Confirm the date range and dtype
print(f"Dtype of DATE_REPORTED: {crime_incident_df['DATE_REPORTED'].dtype}")
print(f"Date range: {crime_incident_df['DATE_REPORTED'].min()} to {crime_incident_df['DATE_REPORTED'].max()}")

Dtype of DATE_REPORTED: datetime64[ns]
Date range: 2017-01-01 00:00:00 to 2024-11-20 00:00:00


# Cleaning Data and Creating new DF

In [19]:
# Select only the specified columns
columns_to_keep = ['HIGHEST_NIBRS_DESCRIPTION', 'CLEARANCE_STATUS', 'DATE_REPORTED', 'CMPD_PATROL_DIVISION']
cleaned_data = crime_incident_df[columns_to_keep]

# Drop rows with missing values in the selected columns
cleaned_data = cleaned_data.dropna(subset=columns_to_keep)
cleaned_data.head()



,HIGHEST_NIBRS_DESCRIPTION,CLEARANCE_STATUS,DATE_REPORTED,CMPD_PATROL_DIVISION
0,Suicide,Open,2023-08-17,Freedom
1,Missing Person,Exceptionally Cleared,2022-02-06,South
2,Other Unlisted Non-Criminal,Exceptionally Cleared,2023-12-18,Providence
3,Theft of Motor Vehicle Parts from Vehicle,Open,2020-10-11,Steele Creek
4,Intimidation,Exceptionally Cleared,2017-04-01,Steele Creek


In [20]:
# Filter rows where 'HIGHEST_NIBRS_DESCRIPTION' is either 'Aggrevated Assualt', 'Simple Assault', or 'Intimidation'
filtered_data = cleaned_data[cleaned_data['HIGHEST_NIBRS_DESCRIPTION'].isin(['Aggrevated Assualt','Simple Assault', 'Intimidation'])]
filtered_data.head()


,HIGHEST_NIBRS_DESCRIPTION,CLEARANCE_STATUS,DATE_REPORTED,CMPD_PATROL_DIVISION
4,Intimidation,Exceptionally Cleared,2017-04-01,Steele Creek
12,Intimidation,Exceptionally Cleared,2023-04-14,South
36,Simple Assault,Open,2024-04-06,Central
38,Simple Assault,Open,2024-02-02,Providence
56,Simple Assault,Cleared by Arrest,2017-11-17,Westover


In [21]:
filtered_data['DATE_REPORTED']

4        2017-04-01
12       2023-04-14
36       2024-04-06
38       2024-02-02
56       2017-11-17
            ...    
708237   2017-04-28
708242   2021-03-23
708249   2024-01-05
708262   2018-02-08
708286   2022-07-11
Name: DATE_REPORTED, Length: 83037, dtype: datetime64[ns]

In [22]:
# Filter rows where 'HIGHEST_NIBRS_DESCRIPTION' is either 'Aggrevated Assualt', 'Simple Assault', or 'Intimidation'
filtered_data_2 = cleaned_data[cleaned_data['HIGHEST_NIBRS_DESCRIPTION'].isin([
        'Missing Person',
        'Theft From Motor Vehicle',
        'Drug/Narcotic Violations', 
        'Burglary/B&E',
        'Shoplifting',
        'Damage/Vandalism Of Property', 
        'Motor Vehicle Theft'
       ])]

filtered_data_2.head()

,HIGHEST_NIBRS_DESCRIPTION,CLEARANCE_STATUS,DATE_REPORTED,CMPD_PATROL_DIVISION
1,Missing Person,Exceptionally Cleared,2022-02-06,South
6,Theft From Motor Vehicle,Unfounded,2017-08-02,Independence
10,Drug/Narcotic Violations,Cleared by Arrest,2017-09-26,Eastway
15,Burglary/B&E,Open,2019-12-16,South
16,Theft From Motor Vehicle,Open,2023-08-05,North


In [23]:
# Get the number of rows and columns
num_rows, num_columns = filtered_data.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


Number of rows: 83037
Number of columns: 4


In [24]:
# Get the number of rows and columns
num_rows, num_columns = filtered_data_2.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")


Number of rows: 268947
Number of columns: 4


In [25]:
# Save the cleaned data to a new CSV file (optional)
filtered_data.to_csv('CMPD_VIOLENT_DATES.csv', index=False)

# Display basic information about the cleaned dataset
print(cleaned_data.info())
print(cleaned_data.head())

<class 'pandas.core.frame.DataFrame'>
Index: 707653 entries, 0 to 708287
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   HIGHEST_NIBRS_DESCRIPTION  707653 non-null  object        
 1   CLEARANCE_STATUS           707653 non-null  object        
 2   DATE_REPORTED              707653 non-null  datetime64[ns]
 3   CMPD_PATROL_DIVISION       707653 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 27.0+ MB
None
                   HIGHEST_NIBRS_DESCRIPTION       CLEARANCE_STATUS  \
0                                    Suicide                   Open   
1                             Missing Person  Exceptionally Cleared   
2                Other Unlisted Non-Criminal  Exceptionally Cleared   
3  Theft of Motor Vehicle Parts from Vehicle                   Open   
4                               Intimidation  Exceptionally Cleared   

  DATE_REPORTED CMP

In [26]:
# Save the cleaned data to a new CSV file (optional)
filtered_data_2.to_csv('CMPD_NON_VIOLENT_DATES.csv', index=False)

# Display basic information about the cleaned dataset
print(cleaned_data.info())
print(cleaned_data.head())

<class 'pandas.core.frame.DataFrame'>
Index: 707653 entries, 0 to 708287
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   HIGHEST_NIBRS_DESCRIPTION  707653 non-null  object        
 1   CLEARANCE_STATUS           707653 non-null  object        
 2   DATE_REPORTED              707653 non-null  datetime64[ns]
 3   CMPD_PATROL_DIVISION       707653 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 27.0+ MB
None
                   HIGHEST_NIBRS_DESCRIPTION       CLEARANCE_STATUS  \
0                                    Suicide                   Open   
1                             Missing Person  Exceptionally Cleared   
2                Other Unlisted Non-Criminal  Exceptionally Cleared   
3  Theft of Motor Vehicle Parts from Vehicle                   Open   
4                               Intimidation  Exceptionally Cleared   

  DATE_REPORTED CMP